# Ejercicio aplicado de DataFrames y Spark SQL

In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Df')\
        .getOrCreate()

In [7]:
## Importa el csv de "data/WorldCupPlayers.csv"
## Visualiza los datos
# File path
data_load = r'C:\Users\HP\Documents\MCD\Semestres\1ro\Manejo_de_Grandes_Bases_de_Datos\ProcesBigdata24B\Process_Big_Data\Notebooks\0. Ejercicios\Soluciones_Ejercicios\data\WorldCupPlayers.csv'

# Load the CSV into a DataFrame
df = (spark.read
          .format("csv")
          .option('header', 'true')
          .load(data_load))  # Use the variable here

# Visualize the data
df.show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 

In [12]:
## ¿que tipo de datos contiene cada variable?
df.describe().toPandas()

,summary,RoundID,MatchID,Team Initials,Coach Name,Line-up,Shirt Number,Player Name,Position,Event
0,count,37784,37784,37784,37784,37784,37784,37784,4143,9069
1,mean,1.105647444595596E7,6.362232957122062E7,None,None,None,10.726021596442939,None,None,None
2,stddev,2.77014365284288E7,1.123915841640783E8,None,None,None,6.960138422882888,None,None,None
3,min,1014,1012,ALG,ACOSTA Nelson (URU),N,0,?URI?I?,C,G1'
4,max,97410600,996,ZAI,ZICO (BRA),S,9,�ZIL,GKC,Y99'


In [13]:
## ¿Cuantos registros hay?
df.count()

37784

In [16]:
## Obtén los principales estadísticos de Position
df.select('Position').describe().show()

+-------+--------+
|summary|Position|
+-------+--------+
|  count|    4143|
|   mean|    NULL|
| stddev|    NULL|
|    min|       C|
|    max|     GKC|
+-------+--------+



In [18]:
## Slecciona y muestra los registros distintos de 'Player Name','Coach Name'
df.select('Player Name', 'Coach Name').distinct().show()

+--------------------+--------------------+
|         Player Name|          Coach Name|
+--------------------+--------------------+
|    Arturo FERNANDEZ| BRU Francisco (ESP)|
|Cayetano CARRERAS...|DURAND LAGUNA Jos...|
|  Ernesto MASCHERONI|SUPPICI Alberto (...|
|          Aziz FAHMY|   McREA James (SCO)|
|        Gyula POLGAR|    NADAS Odon (HUN)|
|  Ernesto ALBARRACIN|PASCUCCI Felipe (...|
| Armando CASTELLAZZI|POZZO Vittorio (ITA)|
|     Jaroslav BOUCEK|   PETRU Karel (TCH)|
|           Erwin NYC|  KALUZA Jozef (POL)|
|     Stanislaw BARAN|  KALUZA Jozef (POL)|
|     Fernando ROLDAN|BUCCIARDI Arturo ...|
|            Joe MACA|  JEFFREY Bill (SCO)|
|               INDIO|  MOREIRA Zeze (BRA)|
|      Rene DEREUDDRE|PIBAROT Pierre (FRA)|
|    Anton MALATINSKY|    CEJP Josef (TCH)|
|    Alberto MARIOTTI|LORENZO Juan Carl...|
|  Alfredo DI STEFANO|HERRERA Helenio (...|
|             FIDELIS| FEOLA Vicente (BRA)|
|     Stoyan YORDANOV|BOZHKOV Stefan (BUL)|
|      Wim RIJSBERGEN| MICHELS R

In [20]:
## ¿Cuantos partidos con el ID de 1096 ha habido?
df.filter(df["RoundID"] == 1096).count()

0

In [22]:
## Muestra los datos donde la posicion haya sido C y el evento sea G40
df.filter(df["Position"] == 'C').filter(df["Event"] == 'G40').show()

+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+
|RoundID|MatchID|Team Initials|Coach Name|Line-up|Shirt Number|Player Name|Position|Event|
+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+
+-------+-------+-------------+----------+-------+------------+-----------+--------+-----+



In [28]:
## Utiliza Spark SQL para mostras los registros donde el MatchID sea mayor o igual a 20
df.createOrReplaceTempView("temp_table")

# Execute SQL-Like query.
spark.sql('''
SELECT * 
FROM temp_table 
WHERE MatchID >= 20''').show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 